Напишите парсер на Python, который соберет данные с сайта https://www.585zolotoy.ru/ в категории товаров "Браслеты с камнями".

In [100]:
!pip install selenium

In [101]:
from selenium import webdriver

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(options=options)
driver.get(' https://httpbin.org/ip ')
print(driver.page_source)
driver.quit()

<html><head><meta name="color-scheme" content="light dark"><meta charset="utf-8"></head><body><pre>{
  "origin": "34.138.117.170"
}
</pre><div class="json-formatter-container"></div></body></html>


In [106]:
import requests
from bs4 import BeautifulSoup
import csv
import time

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
}

url = 'https://www.585zolotoy.ru/catalog/bracelets/?segment=479998,479999,480000,480001,480002,480003,480004,480005,480006,480009,480014,480016,480024,480025,480027,480028,480029,480035,480039,480040,480044,480046,480047,480049,480053,480054,480057,480059,480065,480067'
response = requests.get(url, headers=headers)

if response.status_code != 200:
    print("Ошибка:", response.status_code)
else:
    soup = BeautifulSoup(response.text, 'html.parser')
    links = soup.find_all('a', class_='product-card__name-link')
    all_data = []

    print(f"Найдено {len(links)} товаров\n")

    for link in links:
        title = link.get_text(strip=True)
        href = ('https://www.585zolotoy.ru' + link['href']).strip()

        try:
            product_response = requests.get(href, headers=headers, timeout=10)
            if product_response.status_code != 200:
                print(f"Ошибка загрузки {href}: {product_response.status_code}")
                continue

            product_soup = BeautifulSoup(product_response.text, 'html.parser')

            # Название
            h1 = product_soup.find('h1')
            title_full = h1.get_text(strip=True) if h1 else title

            # Цена до скидки
            price_old_elem = product_soup.find('span', class_='product-price__simple-old')
            price_old = price_old_elem.get_text(strip=True) if price_old_elem else None

            # Цена после скидки
            price_discount_elem = product_soup.find('strong', class_='product-price__simple-current')
            price_discount = price_discount_elem.get_text(strip=True) if price_discount_elem else None

            # Если нет скидки — берём старую цену
            if not price_discount:
                price_discount = price_old if price_old else "Не указана"

            # Вес изделия
            weight_label = product_soup.find('div', string='Вес изделия')
            weight_elem = weight_label.find_parent('li').find('div', class_='value') if weight_label else None
            weight = weight_elem.get_text(strip=True) if weight_elem else "Не указан"

            # СОХРАНЯЕМ ДАННЫЕ В СПИСОК
            item_data = {
                'Название': title_full,
                'Ссылка': href,
                'Цена до скидки': price_old or "Не указана",
                'Цена после скидки': price_discount,
                'Вес изделия': weight
            }
            all_data.append(item_data)

            # Выводим для контроля
            print(f"Название: {title_full}")
            print(f"Ссылка: {href}")
            print(f"Цена до скидки: {price_old or 'Не указана'}")
            print(f"Цена после скидки: {price_discount}")
            print(f"Вес изделия: {weight}")
            print("-" * 60)

            time.sleep(1)

        except Exception as e:
            print(f"Ошибка при обработке {href}: {e}")

    # что попало в all_data
    print(f"\n Собрано {len(all_data)} товаров:")
    for data in all_data:
        print(data)

    # CSV
    with open('braslety_585.csv', 'w', encoding='utf-8', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=['Название', 'Ссылка', 'Цена до скидки', 'Цена после скидки', 'Вес изделия'])
        writer.writeheader()
        writer.writerows(all_data)

    print("\n Данные сохранены в braslety_585.csv")

Найдено 60 товаров

Название: Серебряный браслет с фианитами
Ссылка: https://www.585zolotoy.ru/catalog/products/7880980/
Цена до скидки: 8 380 ₽
Цена после скидки: 4 190 ₽
Вес изделия: от 11.47 гр
------------------------------------------------------------
Название: Серебряный браслет с гранатом и фианитами
Ссылка: https://www.585zolotoy.ru/catalog/products/3926433/
Цена до скидки: 14 980 ₽
Цена после скидки: 3 745 ₽
Вес изделия: от 4.84 гр
------------------------------------------------------------
Название: Золотой браслет с ониксом
Ссылка: https://www.585zolotoy.ru/catalog/products/1312393/
Цена до скидки: 145 980 ₽
Цена после скидки: 36 495 ₽
Вес изделия: от 3.63 гр
------------------------------------------------------------
Название: Серебряный браслет с имитацией кварца и фианитами
Ссылка: https://www.585zolotoy.ru/catalog/products/4356682/
Цена до скидки: 12 980 ₽
Цена после скидки: 3 245 ₽
Вес изделия: от 5.12 гр
------------------------------------------------------------
Н

In [ ]:
import csv
csv_filename = 'braslety_585.csv'
with open(csv_filename, 'w', encoding='utf-8', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=['Название', 'Ссылка', 'Цена до скидки', 'Цена со скидкой', 'Вес изделия'])
    writer.writeheader()  # Записываем заголовки
    writer.writerows(all_data)  # Записываем все строки

print(f"Данные сохранены в файл: {csv_filename}")

Данные сохранены в файл: braslety_585.csv


In [107]:
!ls

braslety_585.csv  sample_data


In [108]:
from google.colab import files
files.download('braslety_585.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [109]:
import pandas as pd

# Читаем CSV файл
df = pd.read_csv('braslety_585.csv')

# Показываем все строки целиком (без обрезки)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# Выводим таблицу
print("Содержимое файла braslety_585.csv:")
print(df)

Содержимое файла braslety_585.csv:
                                                   Название  \
0                            Серебряный браслет с фианитами   
1                 Серебряный браслет с гранатом и фианитами   
2                                 Золотой браслет с ониксом   
3         Серебряный браслет с имитацией кварца и фианитами   
4                   Браслет из жемчуга с золотыми вставками   
5                       Браслет из жемчуга с золотым замком   
6                            Серебряный браслет с фианитами   
7              Золотой браслет с топазами Swiss и фианитами   
8                               Золотой браслет с фианитами   
9                            Золотой браслет с бриллиантами   
10                              Золотой браслет с фианитами   
11                              Золотой браслет с фианитами   
12                              Золотой браслет с фианитами   
13                           Золотой браслет с бриллиантами   
14                  

In [99]:
# Проверка: есть ли данные?
print("🔍 Количество собранных товаров:", len(all_data))

if len(all_data) == 0:
    print("⚠️ Предупреждение: данные не собраны. Проверьте логи выше.")
else:
    # Только если данные есть — сохраняем
    with open('braslety_585.csv', 'w', encoding='utf-8', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=['Название', 'Ссылка', 'Цена до скидки', 'Цена со скидкой', 'Вес изделия'])
        writer.writeheader()
        writer.writerows(all_data)

    print("✅ Данные успешно сохранены в braslety_585.csv")

🔍 Количество собранных товаров: 0
⚠️ Предупреждение: данные не собраны. Проверьте логи выше.
